#### Qu'est ce que SQLAlchemy ?

SQLAlchemy est un ORM qui permet de faire des requetes sur une base de données. Il permet de se connecter et requeter une base de données relationnelle de manière sécuriser et en python. 

In [2]:
from sqlalchemy import create_engine, text, Table, Column, Integer,Float, String, MetaData
import pandas as pd

In [3]:
# connexion à la db mysql
def connexion(user, password, host, port, database):
    try:
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}")
        conn = engine.connect()

        conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database}"))
        print(f"Database {database} created")
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
        conn = engine.connect()
    
        return conn, engine
    except Exception as e:
        print(e)
        return None, None

In [4]:
# ouverture des fichiers 
def open_file(file):
    try: 
        df = pd.read_csv(file)
        return df
    except Exception as e:
        print(e)
        return None

In [5]:
# création de la table
def infer_sqlalchemy_type(dtype):
    """ Map pandas dtype to SQLAlchemy's types """
    if "int" in dtype.name:
        return Integer
    elif "float" in dtype.name:
        return Float
    elif "object" in dtype.name:
        return String(255)
    else:
        return String(255)

In [6]:
# Create table 
def create_table(df, engine, table_name):
    metadata = MetaData()

    columns = [Column(name, infer_sqlalchemy_type(dtype)) for name, dtype in df.dtypes.items()]
    table = Table(table_name, metadata, *columns)

    table.create(engine)

In [7]:
# Connexion à la base de données mariadb et création de la base de données brief2
conn, engine = connexion("root", "password", "localhost", "3306", "brief2_csv")

Database brief2 created


/tmp/ipykernel_69388/2642612893.py:7: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database}"))


In [8]:
df_client = pd.read_csv("data/data-csv/data-csv/clients.csv", sep=";")
df_client.drop_duplicates(inplace=True)
df_product = pd.read_csv("data/data-csv/data-csv/produits_sous-categorie.csv", sep=";")
df_product.drop_duplicates(inplace=True)
df_vente = pd.read_csv("data/data-csv/data-csv/ventes.csv", sep=";")
df_vente.drop_duplicates(inplace=True)

df_list = [df_client, df_product, df_vente]
df_name = ["clients", "produits", "ventes"]

In [9]:
for df, df_name in zip(df_list, df_name):
    # Si la table n'existe pas 
    if not engine.dialect.has_table(conn, df_name):
        create_table(df, engine, df_name)
        print(f"Table {df_name} created")
    print(f"Table {df_name} already exists")

Table clients already exists
Table produits already exists
Table ventes already exists


In [11]:
df_vente.to_sql('ventes', con=engine, if_exists='replace', index=False)
df_product.to_sql('produits', con=engine, if_exists='replace', index=False)
df_client.to_sql('clients', con=engine, if_exists='replace', index=False)

8600

In [12]:
# Connexion à la base de données mariadb et création de la base de données brief2
conn, engine = connexion("root", "password", "localhost", "3306", "brief2_json")

Database brief2_json created


In [18]:
df_client = pd.read_json("data/data-json/data-json/clients.json")
df_client.drop_duplicates(inplace=True)
df_product = pd.read_json("data/data-json/data-json/produits_sous-categorie.json")
df_product.drop_duplicates(inplace=True)
df_vente = pd.read_json("data/data-json/data-json/ventes.json")
df_vente.drop_duplicates(inplace=True)

In [16]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8600 entries, 0 to 324624
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8600 non-null   object
 1   sex        8600 non-null   object
 2   birth      8600 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 268.8+ KB


In [ ]:
df_list = [df_client, df_product, df_vente]
df_name = ["clients", "produits", "ventes"]